<a href="https://colab.research.google.com/github/ipeirotis/datasets/blob/master/notebooks/NYPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NYPD Dataset

Dataset description at 
https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i



| Column | Description |
|--------|-------------------|
| CMPLNT_NUM |  Randomly generated persistent ID for each complaint  |  
| ADDR_PCT_CD |  The precinct in which the incident occurred |  
| BORO |  The name of the borough in which the incident occurred |  
| CMPLNT_FR_DT |  Exact date of occurrence for the reported event (or starting date of occurrence, if CMPLNT_TO_DT exists) |  
| CMPLNT_FR_TM |  Exact time of occurrence for the reported event (or starting time of occurrence, if CMPLNT_TO_TM exists) |  
| CMPLNT_TO_DT |  Ending date of occurrence for the reported event, if exact time of occurrence is unknown |  
| CMPLNT_TO_TM |  Ending time of occurrence for the reported event, if exact time of occurrence is unknown |  
| CRM_ATPT_CPTD_CD |  Indicator of whether crime was successfully completed or attempted, but failed or was interrupted prematurely |  
| HADEVELOPT |  Name of NYCHA housing development of occurrence, if applicable |  
| HOUSING_PSA |  Development Level Code |  
| JURISDICTION_CODE |  Jurisdiction responsible for incident. Either internal, like Police(0), Transit(1), and Housing(2); or external(3), like Correction, Port Authority, etc. |  
| JURIS_DESC |  Description of the jurisdiction code |  
| KY_CD |  Three digit offense classification code |  
| LAW_CAT_CD |  Level of offense: felony, misdemeanor, violation  |  
| LOC_OF_OCCUR_DESC |  Specific location of occurrence in or around the premises; inside, opposite of, front of, rear of |  
| OFNS_DESC |  Description of offense corresponding with key code |  
| PARKS_NM |  Name of NYC park, playground or greenspace of occurrence, if applicable (state parks are not included) |  
| PATROL_BORO |  The name of the patrol borough in which the incident occurred |  
| PD_CD |  Three digit internal classification code (more granular than Key Code) |  
| PD_DESC |  Description of internal classification corresponding with PD code (more granular than Offense Description) |  
| PREM_TYP_DESC |  Specific description of premises; grocery store, residence, street, etc. |  
| RPT_DT |  Date event was reported to police  |  
| STATION_NAME |  Transit station name |  
| SUSP_AGE_GROUP |  Suspect’s Age Group |  
| SUSP_RACE |  Suspect’s Race Description |  
| SUSP_SEX |  Suspect’s Sex Description |  
| TRANSIT_DISTRICT |  Transit district in which the offense occurred. |  
| VIC_AGE_GROUP |  Victim’s Age Group |  
| VIC_RACE |  Victim’s Race Description |  
| VIC_SEX |  Victim’s Sex Description |  
| X_COORD_CD |  X-coordinate for New York State Plane Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104) |  
| Y_COORD_CD |  Y-coordinate for New York State Plane Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104) |  
| Latitude |  Midblock Latitude coordinate for Global Coordinate System, WGS 1984, decimal degrees (EPSG 4326)  |  
| Longitude |  Midblock Longitude coordinate for Global Coordinate System, WGS 1984, decimal degrees (EPSG 4326) |


In [1]:
import pandas as pd
import numpy as np

In [2]:
# We load everything as an object/string, because some data types (e.g., some IDs)
# are recognized as decimals, and it is a mess to restore them back
# So we will do all the conversions ourselves later on

# From https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i/data
!curl 'https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD' -o nypd.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2565M    0 2565M    0     0  4843k      0 --:--:--  0:09:02 --:--:-- 5152k


In [3]:
%%time
df = pd.read_csv('nypd.csv', low_memory = True, dtype='object')

CPU times: user 48.5 s, sys: 7.25 s, total: 55.8 s
Wall time: 55.6 s


In [ ]:
# We load directly from the URL
# url = 'https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD'
# df = pd.read_csv(url, low_memory = True, dtype='object')


In [4]:
len(df)

8357267

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8357267 entries, 0 to 8357266
Data columns (total 35 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   CMPLNT_NUM         object
 1   CMPLNT_FR_DT       object
 2   CMPLNT_FR_TM       object
 3   CMPLNT_TO_DT       object
 4   CMPLNT_TO_TM       object
 5   ADDR_PCT_CD        object
 6   RPT_DT             object
 7   KY_CD              object
 8   OFNS_DESC          object
 9   PD_CD              object
 10  PD_DESC            object
 11  CRM_ATPT_CPTD_CD   object
 12  LAW_CAT_CD         object
 13  BORO_NM            object
 14  LOC_OF_OCCUR_DESC  object
 15  PREM_TYP_DESC      object
 16  JURIS_DESC         object
 17  JURISDICTION_CODE  object
 18  PARKS_NM           object
 19  HADEVELOPT         object
 20  HOUSING_PSA        object
 21  X_COORD_CD         object
 22  Y_COORD_CD         object
 23  SUSP_AGE_GROUP     object
 24  SUSP_RACE          object
 25  SUSP_SEX           object
 26  TRANSIT_DISTRI

## Data Cleaning

In [6]:
# These columns are redundant
to_drop = ['Lat_Lon','X_COORD_CD','Y_COORD_CD']
# We created the CMPLNT_FR and CMPLNT_TO columns
# We have the longitude and latitude so the other coordinates are not needed
df = df.drop(to_drop, axis='columns')

###  CMPLNT_NUM         object   

In [18]:
df.CMPLNT_NUM = pd.to_numeric(df.CMPLNT_NUM, errors="coerce")
df = df[~df.CMPLNT_NUM.isna()]
# Drop cases with duplicated complaint numbers
key_cnt = df.CMPLNT_NUM.value_counts()
key_cnt [ key_cnt>1 ]
df = df[ ~df.CMPLNT_NUM.isin( key_cnt [ key_cnt>1 ].index.values ) ]

In [19]:
len(df)

8339376

In [20]:
df.CMPLNT_NUM = df.CMPLNT_NUM.astype('int32')

In [21]:
df.CMPLNT_NUM

0          469069650
1          629841380
2          918597562
3          224389328
4          303540290
             ...    
8357262    335059839
8357263    149693535
8357264    538367318
8357265    719675182
8357266    711178036
Name: CMPLNT_NUM, Length: 8339376, dtype: int32

### CMPLNT_FR_DT       object
### CMPLNT_FR_TM       object
### CMPLNT_TO_DT       object
### CMPLNT_TO_TM       object

In [23]:
# There are a few rows that contain year 1015, 1016, ... that trigger an error during date conversion
# We replace all years written as 10XX with 20XX
# Note the usage of regular expressions
df.CMPLNT_FR_DT.replace(to_replace = '(\d\d)/(\d\d)/10(\d\d)', value=r'\1/\2/20\3', inplace = True, regex=True )
df.CMPLNT_TO_DT.replace(to_replace = '(\d\d)/(\d\d)/10(\d\d)', value=r'\1/\2/20\3', inplace = True, regex=True )

# Similarly, a few hours are written as 24:00:00, which also triggers errors.
# We fix these hours
df.CMPLNT_FR_TM.replace(to_replace = '24:00:00', value='00:00:00', inplace = True)
df.CMPLNT_TO_TM.replace(to_replace = '24:00:00', value='00:00:00', inplace = True)

# Convert the two separate date and time columns into single datetime columns
df['CMPLNT_FR'] = pd.to_datetime(df.CMPLNT_FR_DT + ' ' + df.CMPLNT_FR_TM, format='%m/%d/%Y %H:%M:%S', cache=True, errors="coerce")
df['CMPLNT_TO'] = pd.to_datetime(df.CMPLNT_TO_DT + ' ' + df.CMPLNT_TO_TM, format='%m/%d/%Y %H:%M:%S', cache=True, errors="coerce")

# These columns are redundant
to_drop = ['CMPLNT_FR_DT','CMPLNT_TO_DT','CMPLNT_FR_TM','CMPLNT_TO_TM']
# We created the CMPLNT_FR and CMPLNT_TO columns
# We have the longitude and latitude so the other coordinates are not needed
df = df.drop(to_drop, axis='columns')

In [24]:
df.CMPLNT_FR.isnull().sum()

701

In [25]:
df.CMPLNT_TO.isnull().sum()

1781535

In [26]:
df = df [ ~df.CMPLNT_FR.isnull() ]

###  ADDR_PCT_CD        object

In [27]:

df.ADDR_PCT_CD.replace(to_replace = '-99', value='99', inplace = True)
# df = df [ ~df.ADDR_PCT_CD.isnull() ]
df.ADDR_PCT_CD = pd.Categorical(df.ADDR_PCT_CD)

###  RPT_DT             object

In [28]:


# Convert RPT_DT to date
df.RPT_DT = pd.to_datetime(df.RPT_DT, format="%m/%d/%Y", cache=True)

### 7   KY_CD              object
### 8   OFNS_DESC          object

In [29]:
df.OFNS_DESC.replace(to_replace = 'KIDNAPPING', value='KIDNAPPING & RELATED OFFENSES', inplace = True)
df.OFNS_DESC.replace(to_replace = 'KIDNAPPING AND RELATED OFFENSES', value='KIDNAPPING & RELATED OFFENSES', inplace = True)

df.OFNS_DESC.replace(to_replace = 'AGRICULTURE & MRKTS LAW-UNCLASSIFIED', value='OTHER STATE LAWS (NON PENAL LAW)', inplace = True)
df.OFNS_DESC.replace(to_replace = 'OTHER STATE LAWS (NON PENAL LA', value='OTHER STATE LAWS (NON PENAL LAW)', inplace = True)

df.OFNS_DESC.replace(to_replace = 'ENDAN WELFARE INCOMP', value='OFFENSES RELATED TO CHILDREN', inplace = True)

df.OFNS_DESC.replace(to_replace = 'THEFT OF SERVICES', value='OTHER OFFENSES RELATED TO THEF', inplace = True)

df.OFNS_DESC.replace(to_replace = 'NYS LAWS-UNCLASSIFIED VIOLATION', value='OTHER STATE LAWS', inplace = True)

df.OFNS_DESC.replace(to_replace = 'FELONY SEX CRIMES', value='SEX CRIMES', inplace = True)

df.loc[df.KY_CD=='120','OFNS_DESC'] ='CHILD ABANDONMENT/NON SUPPORT'

df.loc[df.KY_CD=='125','OFNS_DESC'] ='NYS LAWS-UNCLASSIFIED FELONY'

offenses = df[ ["KY_CD", "OFNS_DESC"] ].drop_duplicates().dropna()
offenses['KY_CD'] = pd.Categorical(pd.to_numeric(offenses['KY_CD'] ).astype(int))
offenses = offenses.set_index("KY_CD")
offenses = offenses.sort_index()
offenses = offenses.reset_index()
offenses


,KY_CD,OFNS_DESC
0,101,MURDER & NON-NEGL. MANSLAUGHTER
1,102,HOMICIDE-NEGLIGENT-VEHICLE
2,103,"HOMICIDE-NEGLIGENT,UNCLASSIFIE"
3,104,RAPE
4,105,ROBBERY
...,...,...
74,676,NEW YORK CITY HEALTH CODE
75,677,OTHER STATE LAWS
76,678,MISCELLANEOUS PENAL LAW
77,685,ADMINISTRATIVE CODES


In [30]:
df.KY_CD = pd.Categorical(df.KY_CD)

In [31]:
df = df.drop('OFNS_DESC', axis='columns')

### 9   PD_CD              object
### 10  PD_DESC            object

In [32]:



df.loc[df.PD_CD=='694','PD_DESC'] ='INCEST'

df.loc[df.PD_CD=='234','PD_DESC'] ='BURGLARY,UNKNOWN TIME'

internal = df[ ["PD_CD", "PD_DESC"] ].drop_duplicates().dropna()
internal['PD_CD'] = pd.Categorical(pd.to_numeric(internal['PD_CD'] ).astype(int))
internal = internal.set_index("PD_CD")
internal = internal.sort_index()
internal = internal.reset_index()
internal

,PD_CD,PD_DESC
0,100,STALKING COMMIT SEX OFFENSE
1,101,ASSAULT 3
2,102,ASSAULT SCHOOL SAFETY AGENT
3,103,ASSAULT TRAFFIC AGENT
4,104,VEHICULAR ASSAULT (INTOX DRIVE
...,...,...
438,916,LEAVING SCENE-ACCIDENT-PERSONA
439,918,RECKLESS DRIVING
440,922,"TRAFFIC,UNCLASSIFIED MISDEMEAN"
441,969,"TRAFFIC,UNCLASSIFIED INFRACTIO"


In [33]:
df.PD_CD.isnull().sum()

6743

In [34]:
# df = df[~df.PD_CD.isnull()]

In [35]:
df.PD_CD = pd.Categorical(df.PD_CD)

In [36]:
df = df.drop('PD_DESC', axis='columns')

### 11  CRM_ATPT_CPTD_CD   object

In [37]:
df.CRM_ATPT_CPTD_CD.value_counts()

COMPLETED    8198737
ATTEMPTED     139771
Name: CRM_ATPT_CPTD_CD, dtype: int64

In [38]:
df.CRM_ATPT_CPTD_CD = pd.Categorical(df.CRM_ATPT_CPTD_CD)

In [39]:
df.CRM_ATPT_CPTD_CD.isnull().sum()

167

In [ ]:
# df = df [ ~df.CRM_ATPT_CPTD_CD.isnull() ]

### 12  LAW_CAT_CD         object

In [40]:
df.LAW_CAT_CD.value_counts()

MISDEMEANOR    4625621
FELONY         2596201
VIOLATION      1116853
Name: LAW_CAT_CD, dtype: int64

In [41]:
df.LAW_CAT_CD = pd.Categorical(df.LAW_CAT_CD)

### 16  JURIS_DESC         object
### 17  JURISDICTION_CODE  object

In [42]:
df.JURISDICTION_CODE.isnull().sum()

6743

In [43]:
# df = df[ ~df.JURISDICTION_CODE.isnull() ]

jusridiction = df[ ["JURISDICTION_CODE", "JURIS_DESC", ] ].drop_duplicates().dropna()
jusridiction['JURISDICTION_CODE'] = pd.to_numeric(jusridiction['JURISDICTION_CODE'] )
jusridiction['JURISDICTION_CODE'] = jusridiction['JURISDICTION_CODE'].astype(int)
jusridiction = jusridiction.set_index("JURISDICTION_CODE")
jusridiction = jusridiction.sort_index()
jusridiction = jusridiction.reset_index()
jusridiction

,JURISDICTION_CODE,JURIS_DESC
0,0,N.Y. POLICE DEPT
1,1,N.Y. TRANSIT POLICE
2,2,N.Y. HOUSING POLICE
3,3,PORT AUTHORITY
4,4,TRI-BORO BRDG TUNNL
5,6,LONG ISLAND RAILRD
6,7,AMTRACK
7,8,CONRAIL
8,9,STATN IS RAPID TRANS
9,11,N.Y. STATE POLICE


In [44]:
df.JURISDICTION_CODE = pd.Categorical(df.JURISDICTION_CODE)


In [45]:
df = df.drop('JURIS_DESC', axis='columns')

###  13  BORO_NM            object

In [98]:
df.BORO_NM.value_counts()

BROOKLYN         2455184
MANHATTAN        2011169
BRONX            1802202
QUEENS           1672912
STATEN ISLAND     383786
Name: BORO_NM, dtype: int64

In [95]:
df.BORO_NM.replace(to_replace = '(null)', value=np.nan, inplace = True)

In [96]:
df.BORO_NM.isnull().sum()

13422

In [ ]:
# df = df[~df.BORO_NM.isnull()]

In [97]:
df.BORO_NM = pd.Categorical(df.BORO_NM)

### 23  SUSP_AGE_GROUP     object
### 32  VIC_AGE_GROUP      object

In [49]:
# Both columns have a lot of noisy entries. We keep only the dominant groups, and also define an order
df.SUSP_AGE_GROUP = pd.Categorical(df.SUSP_AGE_GROUP, ordered=True, categories=['<18', '18-24',  '25-44', '45-64', '65+'])
df.VIC_AGE_GROUP = pd.Categorical(df.VIC_AGE_GROUP, ordered=True, categories=['<18', '18-24',  '25-44', '45-64', '65+'])


### 24  SUSP_RACE          object
### 25  SUSP_SEX           object

### 33  VIC_RACE           object
### 34  VIC_SEX            object

In [50]:
df.VIC_SEX.isnull().sum()

308

In [51]:
df.VIC_SEX.value_counts()

F    3265846
M    2771335
E    1230399
D    1068991
L       1792
U          4
Name: VIC_SEX, dtype: int64

In [52]:
# df = df[~df.VIC_SEX.isnull()]

df.VIC_SEX.replace(to_replace = 'U', value=np.nan, inplace = True)

In [53]:
df.VIC_RACE.isnull().sum()

388

In [57]:
df.VIC_RACE.value_counts()

UNKNOWN                           2715042
BLACK                             2014636
WHITE                             1410837
WHITE HISPANIC                    1363115
ASIAN / PACIFIC ISLANDER           506211
BLACK HISPANIC                     291472
AMERICAN INDIAN/ALASKAN NATIVE      36974
Name: VIC_RACE, dtype: int64

In [55]:
df.VIC_RACE.replace(to_replace = 'OTHER', value='UNKNOWN', inplace = True)

In [56]:
df.VIC_RACE.replace(to_replace = '(null)', value='UNKNOWN', inplace = True)

In [ ]:
# df = df[~df.VIC_RACE.isnull()]

In [64]:
df.SUSP_SEX.value_counts()

M    2851587
F     879639
Name: SUSP_SEX, dtype: int64

In [59]:
# U is unknown, same is NULL.
df.SUSP_SEX.replace(to_replace = 'U', value=np.nan, inplace = True)
df.SUSP_SEX.replace(to_replace = '(null)', value=np.nan, inplace = True)

In [66]:
df.SUSP_RACE.value_counts()

BLACK                             1753513
UNKNOWN                           1370386
WHITE HISPANIC                     793030
WHITE                              500548
BLACK HISPANIC                     243102
ASIAN / PACIFIC ISLANDER           154544
AMERICAN INDIAN/ALASKAN NATIVE      13583
Name: SUSP_RACE, dtype: int64

In [65]:
# Very small amount of OTHER values
df.SUSP_RACE.replace(to_replace = 'OTHER', value='UNKNOWN', inplace = True)
df.SUSP_RACE.replace(to_replace = '(null)', value='UNKNOWN', inplace = True)


In [67]:
df.SUSP_RACE = pd.Categorical(df.SUSP_RACE)
df.SUSP_SEX = pd.Categorical(df.SUSP_SEX)
df.VIC_RACE = pd.Categorical(df.VIC_RACE)
df.VIC_SEX = pd.Categorical(df.VIC_SEX)

In [68]:
df.dtypes

CMPLNT_NUM                    int32
ADDR_PCT_CD                category
RPT_DT               datetime64[ns]
KY_CD                      category
PD_CD                      category
CRM_ATPT_CPTD_CD           category
LAW_CAT_CD                 category
BORO_NM                    category
LOC_OF_OCCUR_DESC            object
PREM_TYP_DESC                object
JURISDICTION_CODE          category
PARKS_NM                     object
HADEVELOPT                   object
HOUSING_PSA                  object
SUSP_AGE_GROUP             category
SUSP_RACE                  category
SUSP_SEX                   category
TRANSIT_DISTRICT             object
Latitude                     object
Longitude                    object
PATROL_BORO                  object
STATION_NAME                 object
VIC_AGE_GROUP              category
VIC_RACE                   category
VIC_SEX                    category
CMPLNT_FR            datetime64[ns]
CMPLNT_TO            datetime64[ns]
dtype: object

###  14  LOC_OF_OCCUR_DESC  object

In [72]:
df.LOC_OF_OCCUR_DESC.value_counts()

INSIDE         4266458
FRONT OF       1973679
OPPOSITE OF     214938
REAR OF         173249
OUTSIDE           4152
Name: LOC_OF_OCCUR_DESC, dtype: int64

In [71]:
df.LOC_OF_OCCUR_DESC.replace(to_replace = '(null)', value=np.nan, inplace = True)

In [73]:
df.LOC_OF_OCCUR_DESC.isnull().sum()

1706199

In [74]:
df.LOC_OF_OCCUR_DESC = pd.Categorical(df.LOC_OF_OCCUR_DESC)

### Latitude                     object
### Longitude                    object

In [75]:
# !sudo apt-get update
!sudo apt-get install python3-rtree
!sudo pip3 install geopandas descartes shapely ngram # matplotlib==3.1.3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  javascript-common libjs-jquery libjs-underscore libspatialindex-c6
  libspatialindex-dev libspatialindex6
Suggested packages:
  apache2 | lighttpd | httpd
The following NEW packages will be installed:
  javascript-common libjs-jquery libjs-underscore libspatialindex-c6
  libspatialindex-dev libspatialindex6 python3-rtree
0 upgraded, 7 newly installed, 0 to remove and 38 not upgraded.
Need to get 829 kB of archives.
After this operation, 3,054 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 javascript-common all 11 [6,066 B]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libjs-jquery all 3.3.1~dfsg-3 [329 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libjs-underscore all 1.9.1~dfsg-1ubuntu0.20.04.1 [99.5 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/univer

In [76]:
import geopandas as gpd

In [77]:
df.Latitude = pd.to_numeric(df.Latitude, downcast='float')
df.Longitude  = pd.to_numeric(df.Longitude, downcast='float')

In [78]:
%%time
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

CPU times: user 9.36 s, sys: 840 ms, total: 10.2 s
Wall time: 10.8 s


In [79]:
shapefile_url = 'https://data.cityofnewyork.us/api/geospatial/cpf4-rkhq?method=export&format=Shapefile'
df_nyc = gpd.GeoDataFrame.from_file(shapefile_url)

In [81]:
!mkdir -p maps
!curl -s https://s-media.nyc.gov/agencies/dcp/assets/files/zip/data-tools/bytes/nynta2010_23a.zip  -o maps/nynta2010_23a.zip
!cd maps && unzip -o nynta2010_23a.zip
shapefile = f"maps/nynta2010_23a/nynta2010.shp"
df_nyc = gpd.GeoDataFrame.from_file(shapefile)
df_nyc = df_nyc.to_crs(4326)

Archive:  nynta2010_23a.zip
  inflating: nynta2010_23a/nynta2010.shp  
  inflating: nynta2010_23a/nynta2010.dbf  
  inflating: nynta2010_23a/nynta2010.shx  
  inflating: nynta2010_23a/nynta2010.prj  
  inflating: nynta2010_23a/nynta2010.shp.xml  


In [82]:
df_nyc

,BoroCode,BoroName,CountyFIPS,NTACode,NTAName,Shape_Leng,Shape_Area,geometry
0,4,Queens,081,QN08,St. Albans,45401.316803,7.741275e+07,"POLYGON ((-73.75205 40.70523, -73.75174 40.704..."
1,2,Bronx,005,BX28,Van Cortlandt Village,21945.719299,2.566612e+07,"POLYGON ((-73.88705 40.88435, -73.88705 40.884..."
2,4,Queens,081,QN55,South Ozone Park,36708.169305,8.246139e+07,"POLYGON ((-73.80577 40.68293, -73.80552 40.682..."
3,3,Brooklyn,047,BK40,Windsor Terrace,19033.672066,1.404167e+07,"POLYGON ((-73.98017 40.66115, -73.98021 40.661..."
4,3,Brooklyn,047,BK50,Canarsie,43703.609666,8.208968e+07,"MULTIPOLYGON (((-73.88834 40.64671, -73.88835 ..."
...,...,...,...,...,...,...,...,...
190,2,Bronx,005,BX34,Melrose South-Mott Haven North,19589.196431,1.727176e+07,"POLYGON ((-73.90129 40.82048, -73.90160 40.819..."
191,2,Bronx,005,BX39,Mott Haven-Port Morris,35604.790810,4.189861e+07,"MULTIPOLYGON (((-73.89681 40.79581, -73.89694 ..."
192,2,Bronx,005,BX63,West Concourse,28571.879354,1.936642e+07,"POLYGON ((-73.91192 40.84326, -73.91195 40.843..."
193,5,Staten Island,085,SI22,West New Brighton-New Brighton-St. George,66052.593065,5.602857e+07,"POLYGON ((-74.07258 40.63794, -74.07330 40.637..."


In [84]:
%%time
# Match each accident with a neighborhood. 
# Will take ~30 mins to run
# This is done with left join, 
# so we preserve all the data points
# but we know which ones are not matching with the shapefile
gdf.crs = df_nyc.crs
gdf = gpd.sjoin(gdf, df_nyc, how='left')


CPU times: user 2min 10s, sys: 12.4 s, total: 2min 22s
Wall time: 2min 22s


In [ ]:
gdf.dtypes

In [88]:
# We keep only boro_name and ntaname
todrop = [
    'index_right', 'BoroCode', 'CountyFIPS', 
    'Shape_Area', 'Shape_Leng'
]

gdf = gdf.drop(todrop, axis='columns')

# Rename the columns
gdf = gdf.rename({
    'BoroName': 'BOROUGH',
    'NTAName': 'NEIGHBORHOOD',
    'NTACode': 'NEIGHBORHOOD_CODE',
}, axis='columns')

In [89]:
gdf['BOROUGH'] = gdf['BOROUGH'].str.upper()

In [90]:
# Mark as NULL all the lon/lat entries outside the NYC area
gdf.loc[gdf.BOROUGH.isnull(), 'Latitude'] = None
gdf.loc[gdf.BOROUGH.isnull(), 'Longitude'] = None

In [91]:
# Mark as NULL all the lon/lat entries that generate inconsistencies
mask = gdf.query('BOROUGH != BORO_NM and Latitude==Latitude and Longitude==Longitude').CMPLNT_NUM.values
condition = gdf.CMPLNT_NUM.isin(mask)

gdf.loc[condition, 'Latitude'] = None
gdf.loc[condition, 'Longitude'] = None

In [ ]:
# Delete the cases where the reported and detected boroughs are different
# to_delete = sorted(set(to_delete))
# gdf = gdf [ ~gdf.CMPLNT_NUM.isin(to_delete) ]

In [92]:
gdf = gdf.drop('geometry', axis='columns')

In [93]:
df = pd.DataFrame(gdf)

In [99]:
df.BORO_NM.value_counts()

BROOKLYN         2455184
MANHATTAN        2011169
BRONX            1802202
QUEENS           1672912
STATEN ISLAND     383786
Name: BORO_NM, dtype: int64

In [100]:
#Temporarily, we drop these. We will add them back in the future
df.drop(['BOROUGH','NEIGHBORHOOD','NEIGHBORHOOD_CODE'], axis='columns', inplace=True)

### TRANSIT_DISTRICT

In [101]:
df.TRANSIT_DISTRICT.value_counts()


4     30097
2     22663
1     18021
3     17854
20    16537
33    16406
12    13746
11    12832
32    12436
30    11766
34     8031
23     2982
Name: TRANSIT_DISTRICT, dtype: int64

In [102]:
len(df) - df.TRANSIT_DISTRICT.isnull().sum()

183371

In [103]:
df.drop('TRANSIT_DISTRICT', axis='columns', inplace=True)


### PREM_TYP_DESC

In [104]:
df.PREM_TYP_DESC.value_counts()

STREET                        2626997
RESIDENCE - APT. HOUSE        1787734
RESIDENCE-HOUSE                821443
RESIDENCE - PUBLIC HOUSING     618026
OTHER                          219612
                               ...   
COLLEGE/UNIVERSITY                499
DAYCARE FACILITY                  170
TRAMWAY                           151
MOBILE FOOD                       141
REAL ESTATE                       112
Name: PREM_TYP_DESC, Length: 78, dtype: int64

In [105]:
df.PREM_TYP_DESC.isnull().sum()

41958

In [ ]:
# df = df [~df.PREM_TYP_DESC.isnull()]

In [106]:
df.PREM_TYP_DESC = pd.Categorical(df.PREM_TYP_DESC)

In [107]:
df.PARKS_NM.value_counts()

(null)                                                 523240
CENTRAL PARK                                             2066
FLUSHING MEADOWS CORONA PARK                             1764
WASHINGTON SQUARE PARK                                   1320
CONEY ISLAND BEACH & BOARDWALK                           1289
                                                        ...  
O'SULLIVAN PLAZA                                            1
HANCOCK T AND T                                             1
MCCARTHY SQUARE                                             1
UNNAMED PARK ON BRUCKNER EXPRESSWAY & ZEREGA AVENUE         1
RALPH BUNCHE PARK                                           1
Name: PARKS_NM, Length: 1261, dtype: int64

In [108]:
df.PARKS_NM.value_counts().sum()

557272

In [109]:
df.drop('PARKS_NM', axis='columns', inplace=True)



 19  HADEVELOPT         object


In [110]:
df.HADEVELOPT.value_counts()

(null)                                         524917
CASTLE HILL                                      7543
VAN DYKE I                                       6100
MARCY                                            5589
GRANT                                            5375
                                                ...  
FRANKLIN AVENUE III MHOP                            1
BRONXCHESTER                                        1
JEFFERSON                                           1
FOREST HILLS COOP (108TH STREET-62ND DRIVE)         1
1010 EAST 178TH STREET                              1
Name: HADEVELOPT, Length: 280, dtype: int64

In [111]:
df.drop('HADEVELOPT', axis='columns', inplace=True)


 20  HOUSING_PSA        object



In [112]:
df.HOUSING_PSA.value_counts()

(null)    492907
670         8278
887         8154
845         7336
720         7167
           ...  
26470          1
1434           1
732            1
28332          1
51,723         1
Name: HOUSING_PSA, Length: 5121, dtype: int64

In [113]:
df.HOUSING_PSA.value_counts().sum()

1124831

In [114]:
df.drop('HOUSING_PSA', axis='columns', inplace=True)

 30  PATROL_BORO        object


In [115]:
df.PATROL_BORO.value_counts()

PATROL BORO BRONX            1802973
PATROL BORO BKLYN SOUTH      1231422
PATROL BORO BKLYN NORTH      1225265
PATROL BORO MAN SOUTH        1015432
PATROL BORO MAN NORTH         996345
PATROL BORO QUEENS NORTH      870766
PATROL BORO QUEENS SOUTH      805362
PATROL BORO STATEN ISLAND     383910
Name: PATROL_BORO, dtype: int64

In [116]:
df.PATROL_BORO = pd.Categorical(df.PATROL_BORO)

In [117]:
df.PATROL_BORO.isnull().sum()

7200

In [ ]:
# df = df[~df.PATROL_BORO.isnull()] 

 31  STATION_NAME       object

In [122]:
df.STATION_NAME.value_counts()

125 STREET                        8884
14 STREET                         4897
42 ST.-PORT AUTHORITY BUS TERM    4370
34 ST.-PENN STATION               4257
42 ST.-TIMES SQUARE               3581
                                  ... 
DISTRICT 12 OFFICE                  20
DISTRICT 30 OFFICE                  19
DISTRICT 34 OFFICE                  14
DISTRICT 23 OFFICE                   6
OFF-SYSTEM                           4
Name: STATION_NAME, Length: 372, dtype: int64

In [119]:
df.STATION_NAME.replace(to_replace = '(null)', value=np.nan, inplace = True)

In [120]:
df.STATION_NAME.isnull().sum()



8155304

In [123]:
df.drop('STATION_NAME', axis='columns', inplace=True)

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8338675 entries, 0 to 8357266
Data columns (total 22 columns):
 #   Column             Dtype         
---  ------             -----         
 0   CMPLNT_NUM         int32         
 1   ADDR_PCT_CD        category      
 2   RPT_DT             datetime64[ns]
 3   KY_CD              category      
 4   PD_CD              category      
 5   CRM_ATPT_CPTD_CD   category      
 6   LAW_CAT_CD         category      
 7   BORO_NM            category      
 8   LOC_OF_OCCUR_DESC  category      
 9   PREM_TYP_DESC      category      
 10  JURISDICTION_CODE  category      
 11  SUSP_AGE_GROUP     category      
 12  SUSP_RACE          category      
 13  SUSP_SEX           category      
 14  Latitude           float32       
 15  Longitude          float32       
 16  PATROL_BORO        category      
 17  VIC_AGE_GROUP      category      
 18  VIC_RACE           category      
 19  VIC_SEX            category      
 20  CMPLNT_FR          datet

## Data exploration

In this part we check the different values that appear in the columns. When we detect noisy results, we delete the corresponding values. In fact, many of the operations that are performed above, in the 'data cleaning' section, are the result of observations that we make here. 

In [125]:
# Find the unique values in each column
# 
# df.describe(include = [np.object, 'category']).T['unique']
unique = df.describe(include = 'all').T['unique'].sort_values()

<ipython-input-125-f1a1aa553f33>:4: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  unique = df.describe(include = 'all').T['unique'].sort_values()
<ipython-input-125-f1a1aa553f33>:4: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  unique = df.describe(include = 'all').T['unique'].sort_values()
<ipython-input-125-f1a1aa553f33>:4: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  unique = df.descr

In [126]:
unique

CRM_ATPT_CPTD_CD           2
SUSP_SEX                   2
LAW_CAT_CD                 3
VIC_SEX                    5
BORO_NM                    5
LOC_OF_OCCUR_DESC          5
VIC_AGE_GROUP              5
SUSP_AGE_GROUP             5
VIC_RACE                   7
SUSP_RACE                  7
PATROL_BORO                8
JURISDICTION_CODE         25
KY_CD                     75
ADDR_PCT_CD               77
PREM_TYP_DESC             78
PD_CD                    439
RPT_DT                  6209
CMPLNT_FR            2388798
CMPLNT_TO            2651346
CMPLNT_NUM               NaN
Latitude                 NaN
Longitude                NaN
Name: unique, dtype: object

In [127]:
for column in unique.index:
    if unique[column] < 200:
        print(df[column].value_counts())
        print("=====")

COMPLETED    8198737
ATTEMPTED     139771
Name: CRM_ATPT_CPTD_CD, dtype: int64
=====
M    2851587
F     879639
Name: SUSP_SEX, dtype: int64
=====
MISDEMEANOR    4625621
FELONY         2596201
VIOLATION      1116853
Name: LAW_CAT_CD, dtype: int64
=====
F    3265846
M    2771335
E    1230399
D    1068991
L       1792
Name: VIC_SEX, dtype: int64
=====
BROOKLYN         2455184
MANHATTAN        2011169
BRONX            1802202
QUEENS           1672912
STATEN ISLAND     383786
Name: BORO_NM, dtype: int64
=====
INSIDE         4266458
FRONT OF       1973679
OPPOSITE OF     214938
REAR OF         173249
OUTSIDE           4152
Name: LOC_OF_OCCUR_DESC, dtype: int64
=====
25-44    2783842
45-64    1456314
18-24     832418
<18       372119
65+       307012
Name: VIC_AGE_GROUP, dtype: int64
=====
25-44    1261698
18-24     451965
45-64     446197
<18       124697
65+        39377
Name: SUSP_AGE_GROUP, dtype: int64
=====
UNKNOWN                           2715042
BLACK                             2014

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8338675 entries, 0 to 8357266
Data columns (total 22 columns):
 #   Column             Dtype         
---  ------             -----         
 0   CMPLNT_NUM         int32         
 1   ADDR_PCT_CD        category      
 2   RPT_DT             datetime64[ns]
 3   KY_CD              category      
 4   PD_CD              category      
 5   CRM_ATPT_CPTD_CD   category      
 6   LAW_CAT_CD         category      
 7   BORO_NM            category      
 8   LOC_OF_OCCUR_DESC  category      
 9   PREM_TYP_DESC      category      
 10  JURISDICTION_CODE  category      
 11  SUSP_AGE_GROUP     category      
 12  SUSP_RACE          category      
 13  SUSP_SEX           category      
 14  Latitude           float32       
 15  Longitude          float32       
 16  PATROL_BORO        category      
 17  VIC_AGE_GROUP      category      
 18  VIC_RACE           category      
 19  VIC_SEX            category      
 20  CMPLNT_FR          datet

In [129]:
df['PREM_TYP_DESC'].value_counts().count()

78

In [130]:
# All columns, except for the dates and spatial coordinates, are categorical
# Columns with less than a few thousand unique values are good candidates 
# for ENUMs in the database given that the dataset is static.
# Also, in Pandas the internal representation becomes much more efficient
# as the Categoricals are stored as integers and not as strings
for column in unique.index:
    if column == 'RPT_DT':
        continue
    if df[column].value_counts().count() < 1000:
      df[column] = pd.Categorical(df[column])

In [131]:
# With all the proper data typing the dataset went down in size from 1.9Gb+ to 425Mb.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8338675 entries, 0 to 8357266
Data columns (total 22 columns):
 #   Column             Dtype         
---  ------             -----         
 0   CMPLNT_NUM         int32         
 1   ADDR_PCT_CD        category      
 2   RPT_DT             datetime64[ns]
 3   KY_CD              category      
 4   PD_CD              category      
 5   CRM_ATPT_CPTD_CD   category      
 6   LAW_CAT_CD         category      
 7   BORO_NM            category      
 8   LOC_OF_OCCUR_DESC  category      
 9   PREM_TYP_DESC      category      
 10  JURISDICTION_CODE  category      
 11  SUSP_AGE_GROUP     category      
 12  SUSP_RACE          category      
 13  SUSP_SEX           category      
 14  Latitude           float32       
 15  Longitude          float32       
 16  PATROL_BORO        category      
 17  VIC_AGE_GROUP      category      
 18  VIC_RACE           category      
 19  VIC_SEX            category      
 20  CMPLNT_FR          datet

In [132]:
df.memory_usage()

Index                66709400
CMPLNT_NUM           33354700
ADDR_PCT_CD           8341395
RPT_DT               66709400
KY_CD                 8341379
PD_CD                16697414
CRM_ATPT_CPTD_CD      8338799
LAW_CAT_CD            8338807
BORO_NM               8338887
LOC_OF_OCCUR_DESC     8338887
PREM_TYP_DESC         8341403
JURISDICTION_CODE     8339431
SUSP_AGE_GROUP        8338887
SUSP_RACE             8339031
SUSP_SEX              8338799
Latitude             33354700
Longitude            33354700
PATROL_BORO           8339039
VIC_AGE_GROUP         8338887
VIC_RACE              8339031
VIC_SEX               8338887
CMPLNT_FR            66709400
CMPLNT_TO            66709400
dtype: int64

In [133]:
df.dtypes

CMPLNT_NUM                    int32
ADDR_PCT_CD                category
RPT_DT               datetime64[ns]
KY_CD                      category
PD_CD                      category
CRM_ATPT_CPTD_CD           category
LAW_CAT_CD                 category
BORO_NM                    category
LOC_OF_OCCUR_DESC          category
PREM_TYP_DESC              category
JURISDICTION_CODE          category
SUSP_AGE_GROUP             category
SUSP_RACE                  category
SUSP_SEX                   category
Latitude                    float32
Longitude                   float32
PATROL_BORO                category
VIC_AGE_GROUP              category
VIC_RACE                   category
VIC_SEX                    category
CMPLNT_FR            datetime64[ns]
CMPLNT_TO            datetime64[ns]
dtype: object

## Storing in a MySQL database

In [134]:
!sudo pip3 install -U -q PyMySQL sqlalchemy sql_magic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 9.1 MB/s eta 0:00:00


In [135]:
import os
from sqlalchemy import create_engine
from sqlalchemy import text

conn_string = 'mysql+pymysql://{user}:{password}@{host}/?charset=utf8mb4'.format(
    host = 'db.ipeirotis.org', 
    user = 'root',
    password = 'ae6jQniBS5muV27', #os.environ['MYSQL_PASSWORD'],
    encoding = 'utf8mb4')

engine = create_engine(conn_string)


In [140]:
# Query to create a database
db_name = 'nypd'

sql = f"DROP DATABASE IF EXISTS {db_name}"
with engine.connect() as connection:
  connection.execute(text(sql))

# Create a database
sql = f"CREATE DATABASE IF NOT EXISTS {db_name} DEFAULT CHARACTER SET 'utf8mb4'"
with engine.connect() as connection:
  connection.execute(text(sql))


In [141]:

# And lets switch to the database
sql = f"USE {db_name}"
with engine.connect() as connection:
  connection.execute(text(sql))


In [142]:
# In principle, we can let Pandas create the table, but we want to be a bit more predise
# with the data types, and we want to add documentation for each column
# from https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i


sql = '''
CREATE TABLE nypd (
  CMPLNT_NUM int,
  CMPLNT_FR datetime,
  CMPLNT_TO datetime,
  RPT_DT date,
  KY_CD SMALLINT,
  PD_CD SMALLINT,
  CRM_ATPT_CPTD_CD enum('COMPLETED','ATTEMPTED'),
  LAW_CAT_CD enum('FELONY','MISDEMEANOR','VIOLATION'),
  JURISDICTION_CODE SMALLINT,
  BORO_NM enum('BRONX','BROOKLYN','MANHATTAN','QUEENS','STATEN ISLAND'),
  ADDR_PCT_CD SMALLINT,
  LOC_OF_OCCUR_DESC enum('FRONT OF','INSIDE','OPPOSITE OF','OUTSIDE','REAR OF'),
  PATROL_BORO enum('PATROL BORO BRONX', 'PATROL BORO BKLYN SOUTH','PATROL BORO BKLYN NORTH','PATROL BORO MAN SOUTH','PATROL BORO MAN NORTH','PATROL BORO QUEENS NORTH','PATROL BORO QUEENS SOUTH','PATROL BORO STATEN ISLAND'),
  PREM_TYP_DESC varchar(30),
  SUSP_RACE enum('UNKNOWN', 'BLACK', 'WHITE', 'WHITE HISPANIC', 'ASIAN / PACIFIC ISLANDER', 'BLACK HISPANIC', 'AMERICAN INDIAN/ALASKAN NATIVE'),
  VIC_RACE enum('UNKNOWN', 'BLACK', 'WHITE', 'WHITE HISPANIC', 'ASIAN / PACIFIC ISLANDER', 'BLACK HISPANIC', 'AMERICAN INDIAN/ALASKAN NATIVE'),
  SUSP_AGE_GROUP enum('<18', '18-24',  '25-44', '45-64', '65+'),
  VIC_AGE_GROUP enum('<18', '18-24',  '25-44', '45-64', '65+'),
  SUSP_SEX enum('M', 'F'),
  VIC_SEX enum('M', 'F', 'E', 'D', 'L'),
  Latitude double,
  Longitude double,
  PRIMARY KEY (CMPLNT_NUM)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
'''
with engine.connect() as connection:
  connection.execute(text(sql))

In [143]:
# Create a table
# See http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html for the documentation
from tqdm import tqdm
batchsize = 50000
batches = len(df) // batchsize + 1

t = tqdm(range(batches))

for i in t:
    # print("Batch:",i)
    # continue # Cannot execute this on Travis
    start = batchsize * i
    end = batchsize * (i+1)
    df[start:end].to_sql(
        name = 'nypd', 
        schema = db_name, 
        con = engine,
        if_exists = 'append',
        index = False, 
        chunksize = 1000)

100%|██████████| 167/167 [14:05<00:00,  5.06s/it]


In [144]:
sql = "CREATE INDEX ix_lat ON nypd.nypd(Latitude)"
with engine.connect() as connection:
  connection.execute(text(sql))

In [145]:
sql = "CREATE INDEX ix_lon ON nypd.nypd(Longitude)"
with engine.connect() as connection:
  connection.execute(text(sql))

In [146]:
sql = "CREATE INDEX ix_LAW_CAT_CD ON nypd.nypd(LAW_CAT_CD)"
with engine.connect() as connection:
  connection.execute(text(sql))

In [147]:
sql = "CREATE INDEX ix_BORO_NM ON nypd.nypd(BORO_NM)"
with engine.connect() as connection:
  connection.execute(text(sql))

In [148]:
sql = "CREATE INDEX ix_KY_CD ON nypd.nypd(KY_CD)"
with engine.connect() as connection:
  connection.execute(text(sql))

In [149]:
sql = "CREATE INDEX ix_RPT_DT ON nypd.nypd(RPT_DT)"
with engine.connect() as connection:
  connection.execute(text(sql))

In [150]:
sql = "CREATE INDEX ix_CMPLNT_FR ON nypd.nypd(CMPLNT_FR)"
with engine.connect() as connection:
  connection.execute(text(sql))

In [157]:
offenses = offenses[offenses.OFNS_DESC != "(null)"]

In [158]:
offenses

,KY_CD,OFNS_DESC
0,101,MURDER & NON-NEGL. MANSLAUGHTER
1,102,HOMICIDE-NEGLIGENT-VEHICLE
2,103,"HOMICIDE-NEGLIGENT,UNCLASSIFIE"
3,104,RAPE
4,105,ROBBERY
...,...,...
74,676,NEW YORK CITY HEALTH CODE
75,677,OTHER STATE LAWS
76,678,MISCELLANEOUS PENAL LAW
77,685,ADMINISTRATIVE CODES


In [159]:
sql = "DROP TABLE IF EXISTS offense_codes;"
with engine.connect() as connection:
  connection.execute(text(sql))

sql = '''
CREATE TABLE offense_codes (
  KY_CD smallint,
  OFNS_DESC varchar(32),
  PRIMARY KEY (KY_CD)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
'''
with engine.connect() as connection:
  connection.execute(text(sql))

offenses.to_sql(
        name = 'offense_codes', 
        schema = db_name, 
        con = engine,
        if_exists = 'append',
        index = False)

75

In [160]:
sql = "DROP TABLE IF EXISTS jurisdiction_codes;"
with engine.connect() as connection:
  connection.execute(text(sql))
  
sql = '''
CREATE TABLE jurisdiction_codes (
  JURISDICTION_CODE smallint,
  JURIS_DESC varchar(40),
  PRIMARY KEY (JURISDICTION_CODE)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
'''
with engine.connect() as connection:
  connection.execute(text(sql))


jusridiction.to_sql(
        name = 'jurisdiction_codes', 
        schema = db_name, 
        con = engine,
        if_exists = 'append',
        index = False)

25

In [161]:
internal.PD_DESC.str.len().max()


71

In [173]:
internal = internal.query("PD_DESC !=	'CRIMINAL DISPOSAL FIREARM 1 &' ")
internal = internal.query("PD_DESC !=	'UNFINSH FRAME 2' ")
internal = internal.query("PD_DESC !=	'WEAPONS POSSESSION 1 & 2' ")
internal = internal.query("PD_DESC !=	'CRIM POS WEAP 4' ")


In [174]:
internal

,PD_CD,PD_DESC
0,100,STALKING COMMIT SEX OFFENSE
1,101,ASSAULT 3
2,102,ASSAULT SCHOOL SAFETY AGENT
3,103,ASSAULT TRAFFIC AGENT
4,104,VEHICULAR ASSAULT (INTOX DRIVE
...,...,...
438,916,LEAVING SCENE-ACCIDENT-PERSONA
439,918,RECKLESS DRIVING
440,922,"TRAFFIC,UNCLASSIFIED MISDEMEAN"
441,969,"TRAFFIC,UNCLASSIFIED INFRACTIO"


In [175]:
sql = "DROP TABLE IF EXISTS penal_codes;"
with engine.connect() as connection:
  connection.execute(text(sql))

sql = '''
CREATE TABLE penal_codes (
  PD_CD smallint,
  PD_DESC varchar(80),
  PRIMARY KEY (PD_CD)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
'''
with engine.connect() as connection:
  connection.execute(text(sql))


internal.to_sql(
        name = 'penal_codes', 
        schema = db_name, 
        con = engine,
        if_exists = 'append',
        index = False)

439

## TODO

### Add the penal code data as a separate table

`!curl 'https://data.cityofnewyork.us/api/views/qgea-i56i/files/65f25845-1551-4d21-91dc-869c977cd93d?download=true&filename=PDCode_PenalLaw.xlsx' -o PDCode_PenalLaw.xlsx`

### Examine whether to normalize 

The fields 

  
PREM_TYP_DESC    
HADEVELOPT    
PARKS_NM                     

would be better off as foreign keys or enums. They take too much space as strings.